In [1]:
for p in ("Knet","ArgParse","Images","PlyIO")
    Pkg.installed(p) == nothing && Pkg.add(p)
end
using PlyIO
using Knet
using ArgParse
using Images

path = "scapecomp/"
##trial
data0 = load_ply("scapecomp/mesh000.ply")
#atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
datax = data0["vertex"]["x"]
datay = data0["vertex"]["y"]
dataz = data0["vertex"]["z"]
data1 = hcat(datax,datay)
data1 = hcat(data1,dataz)


files = readdir(path)
#println(files[1])
dataset=Any[]
for i in files
    data0 = load_ply(path * i)
    datax = data0["vertex"]["x"]
    datay = data0["vertex"]["y"]
    dataz = data0["vertex"]["z"]
    data1 = hcat(datax,datay)
    data1 = hcat(data1,dataz)
    data1 = reshape(data1,37500,1)
    dataset = push!(dataset,data1)
end

#println(size(dataset))
#dataset = minibatch(dataset, 1; xtype=atype)
#println(length(dataset))
#println(dataset)
##end of trial



const F = Float32

function encode(ϕ, x)
    x = mat(x)
    x = relu.(ϕ[1]*x .+ ϕ[2])
    μ = ϕ[3]*x .+ ϕ[4]
    logσ² = ϕ[5]*x .+ ϕ[6]
    return μ, logσ²
end

function decode(θ, z)
    z = relu.(θ[1]*z .+ θ[2])
    return sigm.(θ[3]*z .+ θ[4])
end

function binary_cross_entropy(x, x̂)
    s = @. x * log(x̂ + F(1e-10)) + (1-x) * log(1 - x̂ + F(1e-10))
    return -mean(s)
end

function output(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    #KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    #KL /= M*28*28 ###1

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return x̂
end

function loss(w, x, nθ)
    θ, ϕ = w[1:nθ], w[nθ+1:end]
    μ, logσ² = encode(ϕ, x)
    nz, M = size(μ)
    σ² = exp.(logσ²)
    σ = sqrt.(σ²)

    KL =  -sum(@. 1 + logσ² - μ*μ - σ²) / 2
    # Normalise by same number of elements as in reconstruction
    KL /= M*28*28 ###1

    z = μ .+ randn!(similar(μ)) .* σ
    x̂ = decode(θ, z)
    BCE = mean(abs2,mat(x)-x̂)
    #BCE = binary_cross_entropy(mat(x), x̂) ####2

    return 1000*BCE #+ KL
end

function aveloss(θ, ϕ, data) #####3
    ls = F(0)
    nθ = length(θ)
    #for (x, y) in data
    for x in data
        ls += loss([θ; ϕ], x, nθ)
    end
    return ls / length(data)
end

function train!(θ, ϕ, data, opt; epochs=1)
    w = [θ; ϕ]
    for epoch=1:epochs
        for (x, y) in data
            dw = grad(loss)(w, x, length(θ))
            update!(w, dw, opt)
        end
    end
    return θ, ϕ
end

function weights(nz, nh; atype=Array{F})
    θ = [  # z->x
        xavier(nh, nz),
        zeros(nh),
        xavier(12500*3, nh), #x
        zeros(12500*3)
        ]
    θ = map(a->convert(atype,a), θ)

    ϕ = [ # x->z
        xavier(nh, 12500*3),
        zeros(nh),
        xavier(nz, nh), #μ
        zeros(nz),
        xavier(nz, nh), #σ
        zeros(nz)
        ]
    ϕ = map(a->convert(atype,a), ϕ)

    return θ, ϕ
end

function main(args="")
    s = ArgParseSettings()
    s.description="Variational Auto Encoder on MNIST dataset."
    s.exc_handler=ArgParse.debug_handler
    @add_arg_table s begin
        ("--seed"; arg_type=Int; default=-1; help="random number seed: use a nonnegative int for repeatable results")
        ("--batchsize"; arg_type=Int; default=100; help="minibatch size")
        ("--epochs"; arg_type=Int; default=10; help="number of epochs for training")
        ("--nh"; arg_type=Int; default=400; help="hidden layer dimension")
        ("--nz"; arg_type=Int; default=40; help="encoding dimention")
        ("--lr"; arg_type=Float64; default=1e-2; help="learning rate")
        ("--atype"; default=(gpu()>=0 ? "KnetArray{F}" : "Array{F}"); help="array type: Array for cpu, KnetArray for gpu")
        ("--infotime"; arg_type=Int; default=2; help="report every infotime epochs")
    end
    isa(args, String) && (args=split(args))
    if in("--help", args) || in("-h", args)
        ArgParse.show_help(s; exit_when_done=false)
        return
    end
    o = parse_args(args, s; as_symbols=true)

    atype = eval(parse(o[:atype]))
    info("using ", atype)
    o[:seed] > 0 && setseed(o[:seed])

    θ, ϕ = weights(o[:nz], o[:nh], atype=atype)
    w = [θ; ϕ]
    opt = optimizers(w, Adam, lr=o[:lr])

    path = "scapecomp/"
    files = readdir(path)
    dataset=Any[]
    for i in files
        data0 = load_ply(path * i)
        datax = data0["vertex"]["x"]
        datay = data0["vertex"]["y"]
        dataz = data0["vertex"]["z"]
        data1 = hcat(datax,datay)
        data1 = hcat(data1,dataz)
        data1=data1/2
        data1[:,2]=data1[:,2]+0.5
        data1[:,3]=data1[:,3]+0.5 
        data1 = reshape(data1,37500,1)
        dataset = push!(dataset,atype(data1))
    end
    #println(typeof(dataset))
    #println(size(dataset))
    #dataset = reshape(dataset,12500,3,1,72)
    #println(size(dataset))
    #dataset = minibatch(dataset,1;xtype=atype())

    #for (d1,d2) in dataset
    #    println(d1)
    #    break
    #end
    #println(size(dataset[1]))
    randomout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    #return reshape(randomout,12500,3)

    #####TRAINING PART#######
    report(epoch) = begin
            #dtrn = minibatch(xtrn, ytrn, o[:batchsize]; xtype=atype)
            #dtst = minibatch(xtst, ytst, o[:batchsize]; xtype=atype)
            println((:epoch, epoch,
                     :trn, aveloss(θ, ϕ, dataset),
                     :length, length(dataset)))
    end

    report(0); tic()
    @time for epoch=1:o[:epochs]
        for x in dataset
            #println(loss(w, atype(reshape(x,37500,1)), length(θ)))           
            dw = grad(loss)(w, atype(reshape(x,37500,1)), length(θ))
            #println(dw[1][1])
            update!(w, dw, opt)
        end
        (epoch % o[:infotime] == 0) && (report(epoch); toc(); tic())
        #println(23)
    end; toq()
    samplein = dataset[1]
    sampleout = output(w, atype(reshape(dataset[1],37500,1)), length(θ))
    sampleout = reshape(sampleout,12500,3)
    return θ, ϕ, reshape(randomout,12500,3), w, sampleout, samplein

end

#return randomout

#end # module

p1, p2, randomout, w, sampleout, samplein = main("--infotime 1 --seed 1 --epochs 300");
println(size(randomout))
sampleout=Array{Float64,2}(sampleout)
writedlm("test5.xyz", sampleout)
#println(randomout)


INFO: using Knet.KnetArray{Float32,N} where N


(:epoch, 0, :trn, 27.27968f0, :length, 26)
(:epoch, 1, :trn, 20.34414f0, :length, 26)
elapsed time: 4.19915355 seconds
(:epoch, 2, :trn, 15.966425f0, :length, 26)
elapsed time: 0.964608867 seconds
(:epoch, 3, :trn, 13.3533325f0, :length, 26)
elapsed time: 0.95219547 seconds
(:epoch, 4, :trn, 11.775665f0, :length, 26)
elapsed time: 1.08767913 seconds
(:epoch, 5, :trn, 10.794313f0, :length, 26)
elapsed time: 0.957412926 seconds
(:epoch, 6, :trn, 10.161933f0, :length, 26)
elapsed time: 0.960526598 seconds
(:epoch, 7, :trn, 9.739662f0, :length, 26)
elapsed time: 0.977113803 seconds
(:epoch, 8, :trn, 9.448096f0, :length, 26)
elapsed time: 1.10973209 seconds
(:epoch, 9, :trn, 9.240554f0, :length, 26)
elapsed time: 0.990130358 seconds
(:epoch, 10, :trn, 9.088732f0, :length, 26)
elapsed time: 0.9589254 seconds
(:epoch, 11, :trn, 8.974946f0, :length, 26)
elapsed time: 1.089314463 seconds
(:epoch, 12, :trn, 8.887813f0, :length, 26)
elapsed time: 0.96330066 seconds
(:epoch, 13, :trn, 8.819811f0, 

(:epoch, 106, :trn, 8.453139f0, :length, 26)
elapsed time: 1.089218389 seconds
(:epoch, 107, :trn, 8.453091f0, :length, 26)
elapsed time: 0.96059415 seconds
(:epoch, 108, :trn, 8.453042f0, :length, 26)
elapsed time: 0.954866305 seconds
(:epoch, 109, :trn, 8.452997f0, :length, 26)
elapsed time: 1.089276857 seconds
(:epoch, 110, :trn, 8.452955f0, :length, 26)
elapsed time: 0.96218813 seconds
(:epoch, 111, :trn, 8.452912f0, :length, 26)
elapsed time: 0.957551936 seconds
(:epoch, 112, :trn, 8.452872f0, :length, 26)
elapsed time: 0.95792441 seconds
(:epoch, 113, :trn, 8.452833f0, :length, 26)
elapsed time: 1.089160532 seconds
(:epoch, 114, :trn, 8.452796f0, :length, 26)
elapsed time: 0.957705707 seconds
(:epoch, 115, :trn, 8.45276f0, :length, 26)
elapsed time: 0.952788721 seconds
(:epoch, 116, :trn, 8.452726f0, :length, 26)
elapsed time: 1.091939222 seconds
(:epoch, 117, :trn, 8.452692f0, :length, 26)
elapsed time: 0.961202853 seconds
(:epoch, 118, :trn, 8.452659f0, :length, 26)
elapsed tim

(:epoch, 210, :trn, 8.4516945f0, :length, 26)
elapsed time: 0.982174897 seconds
(:epoch, 211, :trn, 8.45169f0, :length, 26)
elapsed time: 1.103866611 seconds
(:epoch, 212, :trn, 8.451688f0, :length, 26)
elapsed time: 0.969545604 seconds
(:epoch, 213, :trn, 8.451686f0, :length, 26)
elapsed time: 0.963065183 seconds
(:epoch, 214, :trn, 8.451685f0, :length, 26)
elapsed time: 1.110473939 seconds
(:epoch, 215, :trn, 8.451682f0, :length, 26)
elapsed time: 0.97240201 seconds
(:epoch, 216, :trn, 8.451679f0, :length, 26)
elapsed time: 0.970310743 seconds
(:epoch, 217, :trn, 8.451677f0, :length, 26)
elapsed time: 0.98741735 seconds
(:epoch, 218, :trn, 8.451676f0, :length, 26)
elapsed time: 1.123264728 seconds
(:epoch, 219, :trn, 8.4516735f0, :length, 26)
elapsed time: 0.988469448 seconds
(:epoch, 220, :trn, 8.451672f0, :length, 26)
elapsed time: 0.969815218 seconds
(:epoch, 221, :trn, 8.45167f0, :length, 26)
elapsed time: 1.105976505 seconds
(:epoch, 222, :trn, 8.451668f0, :length, 26)
elapsed t

In [2]:
gpu()

0

In [3]:
samplein=reshape(samplein,12500,3)
samplein=Array{Float64,2}(samplein)
println(maximum(samplein[:,1]), minimum(samplein[:,1]))
println(maximum(samplein[:,2]), minimum(samplein[:,2]))
maximum(samplein[:,3]), minimum(samplein[:,3])
writedlm("samplein.xyz", samplein)
#dataset[1]

0.92395001649856570.007438099943101406
0.58156800270080570.41107550263404846
